## Final Project Submission

Please fill out:
* Student name: Claire Sarraille
* Student pace: self paced
* Scheduled project review date/time:  Monday June 14th 1pm PST
* Instructor name: Jeff Herman
* Project Repo URL: https://github.com/clairesarraille/mod1finproj/tree/main/dsc-mod-1-project-v2-1-online-ds-sp-000-master
* Blog post URL:


Set environment before running this script with: conda activate learn-env
Import datascience libraries:

In [1]:
# Import datascience libraries:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

# Import libraries for fetching and formatting data:
from tmdbv3api import TMDb, Discover, Movie
import os

# Import libraries for performing operations:
from math import ceil

# Load classes from tmdbv3api - a python library for The Movie Database (TMDb) API
tmdb = TMDb()
discover = Discover()
movie = Movie()

# Store my API key in my previously set environment variable as a tmdb.api_key string
priv_api_key = os.environ.get('TMDB_PRIVATE_API_KEY')
tmdb.api_key = priv_api_key

Preview of discover.discover_movies data:

I used this function to quickly scan a sub-set of titles & tweak variables such as:
- 'with_original_language': 'en'

I used a slightly different function to pretty-print samples of all info for a sub-set of movies. See appendix.

In [3]:
def print_sample_tmdb(m, m_r, p, y):
    print()
    print(f"Year is: {y}")
    print(f"Page is: {p +1}")
    print(f"There are {len(m[y])} movies in movies dict for {y}")
    for d in m_r[0:1]:
        print("sample original title: ",(dict(d))['original_title'])
    print()

In [4]:
# Move this to appendix

# import pprint

# def print_sample_tmdb(m, m_r, p, y):  # Pretty-Print preview of discover.discover_movies data
#     print(f"Year is: {y}")
#     print(f"Page is: {p +1}")
#     print(f"There are {len(m[y])} movies in movies dict for {y}")
#     print(f"Here is an example from movies_running:")
#     for index in m_r[0:1]:
#         pprint.pprint(dict(index))
#     print()

Function to extract 100 movies each for years 2000-2019 inclusive, sorted by revenue descending

In [2]:
# Define Constants
# Constants are usually defined on a module level and written in all capital letters with underscores separating words
N_YEAR_RANGE = range(2016, 2022)  # grab 2016-2021 data, inclusive
N_RESULT = 500  # Enter number of results desired for each year
N_RESULT_PAGE = 20  # Discover() class returns 20 results per page
# Num pages = Num results / num results per page, rounded UP using math.ceil
N_PAGES = ceil(N_RESULT / N_RESULT_PAGE)

# Instantiate empty dictionary movies to store  outside of the following loops,
# because you want to use movies after loops are finished executing
movies = {}

# Function to grab data from discover.discover_movies:


def return_discover_movies():
    for year in N_YEAR_RANGE:  # iterate through years 2000-2019 inclusive
        # Keys are integers for each year
        # Values will be lists of tmdbv3api objects, converted to dictionaries
        movies[year] = []
        for page in range(0, N_PAGES):
            # For N_PAGES, create list movies_running, querying results by year, page, & sort revenue desc
            # The objects within movies_running 'tmdbv3api.as_obj.AsObj' objects, and will need to be converted to dictionaries later
            movies_running = discover.discover_movies({
                'sort_by': 'revenue.desc',
                'primary_release_year': str(year),
                # API is 1 indexed - so we grab pages 1 to N_PAGES inclusive
                'page': str(page + 1),
                'vote_count.gte': 1,
                'with_original_language': 'en' # Since our studio is still in R&D mode, we'll focus on movies originally shot in English
            })
            # For each year in our movies dictionary, store the year as a key (datatype list) in dictionary "movies"
            # Then, iterate through 'tmdbv3api.as_obj.AsObj' objects in movies_running for 5 pages, converting each object as a dictionary
            # For each year we will have a list of dictionaries, and each dictionary is a set of information for a single movie
            # .extend used rather than .append in order to extend our list by >1 items at once
            movies[year].extend([dict(tmdb_obj)
                                for tmdb_obj in movies_running])
            # Preview original movie titles:
            # print_sample_tmdb(movies, movies_running, page, year)


# Now we have our dictionary movies, with years 2000-2019 inclusive, each with 100 movies in each list from that year
# We didn't need to return movies in the definition of this function, because movies was declared before the function
return_discover_movies()

In [86]:
# Sanity check film names & make sure movies dictionary contains 2,000 movies
# For this, make setting 'Jupyter: Text Output Limit' = 0 temporarily
movie_count = 0
for year in movies:
    for moo_vie in range(0,len(movies[year])):
        movie_count +=1
        my_string = f"{movies[year][moo_vie]['title']}       {movies[year][moo_vie]['id']}"
        with open("/Users/clairesarraille/git-repos/mod1finproj/dsc-mod-1-project-v2-1-online-ds-sp-000-master/qa_titles.txt", "a+",encoding='utf-8') as file_object:
            # Move read cursor to the start of file.
            file_object.seek(0)
            data = file_object.read(100)
            if len(data) > 0 :
                 file_object.write("\n")
            file_object.write(my_string)


In [ ]:
# While scanning the above list, I found the following that I'll use to develop some clean-up methods:

# These don't look legitimate:
movie_02
813075

movie_01
813064

welcome_home.exe
82418

jji
763071

# There's a double entry for this movie:
Split
358364
Split
381288

# Not in English, this one I later caught when running the analysis to find non-ascii characters:
عتيج
676624


In [36]:
# Sanity check a few film titles and ids fetched from discover.movies with those we get from movie.details():

# Bigfoot vs the Illuminati
# 784281
my_result = movie.details(784281)
print(my_result.title)
print(my_result.id)

# Cheez-It's & Chili
# 802103
my_result = movie.details(802103)
print(my_result.title)
print(my_result.id)

# A Tiny House Christmas
# 796438
my_result = movie.details(796438)
print(my_result.title)
print(my_result.id)

Bigfoot vs the Illuminati
784281
Cheez-It's & Chili
802103
A Tiny House Christmas
796438


In [9]:
movie_count

3000

In [3]:
# Check Keys
movies.keys()

dict_keys([2016, 2017, 2018, 2019, 2020, 2021])

In [4]:
movies_list_of_dict = [movie_d for movie_yr in movies for movie_d in movies[movie_yr]]
df_movies = pd.DataFrame(movies_list_of_dict)
df_movies.head()

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/kvRT3GwcnqGHzPjXIVrVPhUix7Z.jpg,"[12, 28, 878]",271110,en,Captain America: Civil War,"Following the events of Age of Ultron, the col...",86.040,/rAGiXaUfPzY7CDEyNKUofk3Kw2e.jpg,2016-04-27,Captain America: Civil War,False,7.4,18129
1,False,/6t8ES1d12OzWyCGxBeDYLHoaDrT.jpg,"[28, 12, 878]",330459,en,Rogue One: A Star Wars Story,A rogue band of resistance fighters unite for ...,37.495,/5jX3p0apUG5bkMHtnKZch0xpkBS.jpg,2016-12-14,Rogue One: A Star Wars Story,False,7.5,12139
2,False,/wijltj1toRcicHlCn8Cpsp4yPXe.jpg,"[12, 16, 35, 10751]",127380,en,Finding Dory,Dory is reunited with her friends Nemo and Mar...,42.162,/3UVe8NL1E2ZdUZ9EDlKGJY5UzE.jpg,2016-06-16,Finding Dory,False,7.0,9917
3,False,/fGgsIJXTCofFOQydtvWmo9ZIzJX.jpg,"[16, 12, 10751, 35]",269149,en,Zootopia,"Determined to prove herself, Officer Judy Hopp...",60.016,/hlK0e0wAQ3VLuJcsfIYPvb4JVud.jpg,2016-02-11,Zootopia,False,7.7,12988
4,False,/tB2w4m0rW62MTufTjRj0gFLMVBP.jpg,"[10751, 12, 18, 14]",278927,en,The Jungle Book,A man-cub named Mowgli fostered by wolves. Aft...,34.177,/tOEOlmLP71IojeJ91dyJ9Nsb8O8.jpg,2016-04-07,The Jungle Book,False,6.9,6655


In [5]:
# See number of rows and columns, respectively:
df_movies.shape

(3000, 14)

In [ ]:
# Sanity check a few movie ids between df_movies and df_details:
df_movies
df_details


In [6]:
# Create list of movie ids from Pandas df:
movie_id_list = df_movies['id'].tolist()

In [7]:
# Find any movies that would throw an error when used in: movie.details()

my_keys = ['adult', 'budget', 'genres', 'id', 'imdb_id', 'original_language', 'original_title', 'popularity','production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'title', 'keywords']

details_list = []

for id_index in movie_id_list:
    try:
        r = movie.details(id_index)
        details_list.extend({my_key: dict(r).get(my_key) for my_key in my_keys})
    except:
        print(id_index)
        continue

781739


In [8]:
# There is one movie that threw an error. Check it out in our df_movies and then remove from movie_id_list
movie_id_list.remove(781739)


In [9]:
my_keys = ['adult', 'budget', 'genres', 'id', 'imdb_id', 'original_language', 'original_title', 'popularity','production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'title', 'keywords']

details_list = []

for id_index in movie_id_list:
    try:
        r = movie.details(id_index)
        details_list.extend({my_key: dict(r).get(my_key) for my_key in my_keys})
    except:
        print(id_index)
        continue

In [10]:
# small_list_ids = [271110, 330459, 127380, 269149, 278927, 328111, 209112, 259316, 293660, 297761]

my_keys = ['adult', 'budget', 'genres', 'id', 'imdb_id', 'original_language', 'original_title', 'popularity','production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'title', 'keywords']

# In case any of the movies are missing any of the above keys, use:
# dict.get(key, default_value)
# This is one way in which I dealt with missing values that I was receiving in the API

details_list = []
details_list.extend({my_key: dict(movie.details(id_index)).get(my_key) for my_key in my_keys} for id_index in movie_id_list)
print(details_list[0])

{'adult': False, 'budget': 250000000, 'genres': [{'id': 12, 'name': 'Adventure'}, {'id': 28, 'name': 'Action'}, {'id': 878, 'name': 'Science Fiction'}], 'id': 271110, 'imdb_id': 'tt3498820', 'original_language': 'en', 'original_title': 'Captain America: Civil War', 'popularity': 86.04, 'production_companies': [{'id': 420, 'logo_path': '/hUzeosd33nzE5MCNsZxCGEKTXaQ.png', 'name': 'Marvel Studios', 'origin_country': 'US'}], 'production_countries': [{'iso_3166_1': 'US', 'name': 'United States of America'}], 'release_date': '2016-04-27', 'revenue': 1153296293, 'runtime': 147, 'spoken_languages': [{'english_name': 'Romanian', 'iso_639_1': 'ro', 'name': 'Română'}, {'english_name': 'English', 'iso_639_1': 'en', 'name': 'English'}, {'english_name': 'German', 'iso_639_1': 'de', 'name': 'Deutsch'}, {'english_name': 'Russian', 'iso_639_1': 'ru', 'name': 'Pусский'}], 'title': 'Captain America: Civil War', 'keywords': {'keywords': [{'id': 393, 'name': 'civil war'}, {'id': 9663, 'name': 'sequel'}, {'

In [71]:
# Convert details_list to a pandas df:
df_details = pd.DataFrame(details_list)
df_details[0:3]

,adult,budget,genres,id,imdb_id,original_language,original_title,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,title,keywords
0,False,250000000,"[[id, name], [id, name], [id, name]]",271110,tt3498820,en,Captain America: Civil War,86.040,"[[id, logo_path, name, origin_country]]","[[iso_3166_1, name]]",2016-04-27,1153296293,147.0,"[[english_name, iso_639_1, name], [english_nam...",Captain America: Civil War,[keywords]
1,False,200000000,"[[id, name], [id, name], [id, name]]",330459,tt3748528,en,Rogue One: A Star Wars Story,37.495,"[[id, logo_path, name, origin_country]]","[[iso_3166_1, name]]",2016-12-14,1056057273,133.0,"[[english_name, iso_639_1, name]]",Rogue One: A Star Wars Story,[keywords]
2,False,200000000,"[[id, name], [id, name], [id, name], [id, name]]",127380,tt2277860,en,Finding Dory,42.162,"[[id, logo_path, name, origin_country]]","[[iso_3166_1, name]]",2016-06-16,1028570889,97.0,"[[english_name, iso_639_1, name]]",Finding Dory,[keywords]


In [24]:
# Take initial stock:
df_details.index
df_details.info()
df_details.columns
df_details.dtypes


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   adult                 2999 non-null   bool   
 1   budget                2999 non-null   int64  
 2   genres                2999 non-null   object 
 3   id                    2999 non-null   int64  
 4   imdb_id               2283 non-null   object 
 5   original_language     2999 non-null   object 
 6   original_title        2999 non-null   object 
 7   popularity            2999 non-null   float64
 8   production_companies  2999 non-null   object 
 9   production_countries  2999 non-null   object 
 10  release_date          2999 non-null   object 
 11  revenue               2999 non-null   int64  
 12  runtime               2998 non-null   float64
 13  spoken_languages      2999 non-null   object 
 14  title                 2999 non-null   object 
 15  keywords             

False    2999
Name: adult, dtype: int64

In [90]:
# Future: Could check original language by connecting with IMDb & checking language from there - could also use LetterBox
# Detect non-english strings in title and remove them from df_details:
# Set display option so that we can see all rows because we will likely manually review the values:
pd.set_option('display.max_rows', df_details.shape[0]+1)

# Create function to detect non-english strings:
def detect_en(ascii_string):
    try:
        ascii_string.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

column_names = ["id", "title"]
flag_non_en = pd.DataFrame(columns = column_names)
for item in df_details.loc[:, 'title']:
    if isEnglish(item) == False:
        flag_non_en = flag_non_en.append(df_details[['id','title']].loc[df_details['title'] == item])
# print(flag_non_en)

# I identified these movies as being filmed in a language other than English, by conducting internet research on the titles in flag_non_en



In [ ]:
# Create list of movie ids that you want to remove from df_details, based on internet research to see if original language is English:
# 720980      Express Yourself: Die Tänzer der Queen of Pop
# 676624                                               عتيج
# 671272                              Óleos en la oscuridad
# 664648                                          Crisálida
# 692856                                              Avīci
# 691836                      Avante (Ao Vivo em São Paulo)
# 680817                                      Hranice práce
# 676912                                            Sátiros
# 703348                                               عقلي
# 720570                                   maníaco da ponte
# 832125                                  Ghostemane: Ímpio
# 806831                 História Secreta do Pop Brasileiro
# 783999        Crimes de haine en Amérique : l'inquiétante
# 740354                                          Bach-Hông
# 793311                                   Toilette d’amour
# 787619                                           Czyściec
# 832310                                           Verrückt
# 822161                                         ¿Libertad?
# 802434                                              Acéré


# Remove rows with these ids from df_details:
non_eng_ids = [720980, 676624, 671272, 664648, 692856, 691836, 680817, 676912, 703348, 720570, 832125, 806831, 783999, 740354, 793311, 787619, 832310, 822161, 802434]


In [148]:
# Preview the object columns and convert each variable to a separate Boolean column
# Make Keywords column one long string
my_sample = df_details.loc[0:5, 'genres']
type(my_sample)
my_sample
for mystery in my_sample:
    print(type(mystery))
    print(type(mystery[0]))
    print(mystery)
# df_new = pd.json_normalize(df_details, 'genres', ['id','title'])


<class 'list'>
<class 'tmdbv3api.as_obj.AsObj'>
[{'id': 12, 'name': 'Adventure'}, {'id': 28, 'name': 'Action'}, {'id': 878, 'name': 'Science Fiction'}]
<class 'list'>
<class 'tmdbv3api.as_obj.AsObj'>
[{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 878, 'name': 'Science Fiction'}]
<class 'list'>
<class 'tmdbv3api.as_obj.AsObj'>
[{'id': 12, 'name': 'Adventure'}, {'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]
<class 'list'>
<class 'tmdbv3api.as_obj.AsObj'>
[{'id': 16, 'name': 'Animation'}, {'id': 12, 'name': 'Adventure'}, {'id': 10751, 'name': 'Family'}, {'id': 35, 'name': 'Comedy'}]
<class 'list'>
<class 'tmdbv3api.as_obj.AsObj'>
[{'id': 10751, 'name': 'Family'}, {'id': 12, 'name': 'Adventure'}, {'id': 18, 'name': 'Drama'}, {'id': 14, 'name': 'Fantasy'}]
<class 'list'>
<class 'tmdbv3api.as_obj.AsObj'>
[{'id': 12, 'name': 'Adventure'}, {'id': 35, 'name': 'Comedy'}, {'id': 16, 'name': 'Animation'}, {'id': 10751, 'name'

In [ ]:
for obj_in_list in df_details.loc[:, 'genres']:

# Let's call this "custom_sum" as "sum" is a built-in function
def convert_to_dict(val):
    for obj_in_list in df_details.loc[:, 'genres']:
    return row.sum()
    
df['D'] = df.apply(custom_sum, axis=1)

In [121]:
# def convert_to_list_of_dict(val):
#     list_of_dict_tmdb = []
#     for obj_in_list in val:
#         list_of_dict_tmdb.append(dict(obj_in_list))
#     return list_of_dict_tmdb

# df_details['genres_list_dict'] = df_details['genres'].apply(convert_to_dict)

# df_details['genres_list_dict'] 


In [149]:
def convert_to_list_of_dict(val):
    val = [dict(x) for x in val]
    return val

#   movies[year].extend([dict(tmdb_obj) for tmdb_obj in movies_running])
            # Preview original movie titles:
# def convert_to_list_of_dict(val):
#     list_dict_new = []
#     for val_obj_tmdb in val:
#         list_dict_new.append(dict(val_obj_tmdb))
#         return list_dict_new


df_details['genres_list_dict'] = df_details.loc[:,'genres'].apply(convert_to_list_of_dict)

my_sample = df_details.loc[0:5, 'genres_list_dict']
for mystery in my_sample:
    print(type(mystery))
    print(type(mystery[0]))
    print(mystery)

<class 'list'>
<class 'dict'>
[{'id': 12, 'name': 'Adventure'}, {'id': 28, 'name': 'Action'}, {'id': 878, 'name': 'Science Fiction'}]
<class 'list'>
<class 'dict'>
[{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 878, 'name': 'Science Fiction'}]
<class 'list'>
<class 'dict'>
[{'id': 12, 'name': 'Adventure'}, {'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]
<class 'list'>
<class 'dict'>
[{'id': 16, 'name': 'Animation'}, {'id': 12, 'name': 'Adventure'}, {'id': 10751, 'name': 'Family'}, {'id': 35, 'name': 'Comedy'}]
<class 'list'>
<class 'dict'>
[{'id': 10751, 'name': 'Family'}, {'id': 12, 'name': 'Adventure'}, {'id': 18, 'name': 'Drama'}, {'id': 14, 'name': 'Fantasy'}]
<class 'list'>
<class 'dict'>
[{'id': 12, 'name': 'Adventure'}, {'id': 35, 'name': 'Comedy'}, {'id': 16, 'name': 'Animation'}, {'id': 10751, 'name': 'Family'}]


In [143]:
# list_of_dict_tmdb = []
# for list_obj in df_details.loc[:,'genres']:
#     for my_obj in list_obj:
#         list_of_dict_tmdb.append(dict(my_obj))
# list_of_dict_tmdb

In [ ]:
# Drop adult column:
df.drop('DESC', axis=1)

In [ ]:
# Re-order columns

In [ ]:
# Decide what to do with these
# These don't look legitimate:
movie_02
813075

movie_01
813064

welcome_home.exe
82418

jji
763071

# There's a double entry for this movie:
Split
358364
Split
381288